In [1]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [3]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is the purpose of the BGP community attribute, and how is it used?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is the purpose of the BGP community attribute, and how is it used? [/INST]  The Border Gateway Protocol (BGP) community attribute is a feature that allows network operators to assign a specific value to BGP updates, which can be used to control the propagation of routing information within the Internet. The purpose of the BGP community attribute is to provide a way to group BGP updates based on specific criteria, such as the type of network or the origin of the update, and to apply specific policies to these groups.

The BGP community attribute is a 16


In [4]:
import json
import re
from transformers import pipeline

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_true_false.json" 

# Load questions and answers from JSON file
with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

# Initialize text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

llama2_answers = []

# Evaluate generated answers
correct_answers = 0
total_questions = len(questions_data)

for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST] {question} [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(generated_answer)
    llama2_answers.append(generated_answer)

# Extract ground truth answers
ground_truth = [q_dict["answer"] for q_dict in questions_data]

print(ground_truth)

for ground_answer, generated_answer in zip(ground_truth, llama2_answers):
    generated_answer = re.search(r'\[/INST\]\s+(.*?)\.', generated_answer).group(1).strip()
    print(f"Generated Answer: {generated_answer}")
    
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")


[INST] True or False: BGP is a standardized exterior gateway protocol used for routing between autonomous systems (ASes). [/INST]  True. BGP (Border Gateway Protocol) is a standardized exterior gateway protocol used for routing between autonomous systems (ASes).
[INST] True or False: BGP routers exchange their entire routing tables during the BGP peering process. [/INST]  False.

BGP routers do not exchange their entire routing tables during the BGP peering process. Instead, they exchange only the changes to their routing tables, known as "updates," which contain information about the networks that the router can reach.

When two BGP routers establish a peering relationship, they exchange their initial routing tables, but then they only exchange updates as their understanding of the network topology changes. This allows BGP routers to maintain a minimal
[INST] True or False: BGP is a distance-vector routing protocol. [/INST]  False. BGP (Border Gateway Protocol) is a path vector routin

/home/hb/myenv/lib/python3.8/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[INST] True or False: BGP route aggregation involves combining multiple IP prefixes into a single, summarized route. [/INST]  True. BGP route aggregation involves combining multiple IP prefixes into a single, summarized route, which is known as route aggregation or prefix aggregation. This allows for more efficient routing and reduces the number of routes that need to be exchanged between BGP peers.
[INST] True or False: BGP route reflectors help alleviate the full mesh requirement in BGP peerings. [/INST]  True. BGP route reflectors help alleviate the full mesh requirement in BGP peerings by allowing a subset of BGP peers to exchange routing information with each other, rather than requiring a full mesh of peerings between all BGP speakers. This can significantly reduce the number of peerings required and improve the scalability of the BGP system.
[INST] True or False: BGP confederations are used to divide an AS into multiple sub-ASes for better scalability. [/INST]  True. BGP confede

In [5]:
import json

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_multiple_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Generated Answer: When analyzing BGP route propagation, which attribute provides information about the path that the BGP update has traversed?
Choose the correct answer:
a) AS-Path
b) MED (Multi-Exit Discriminator)
c) Local Preference
d) Next Hop

Answer: a) AS-Path

Explanation:

The AS-Path attribute in BGP updates contains a list of autonomous systems (ASes) that the update has traversed. This attribute provides information about the path that the BGP update has taken from the originating AS to the receiving
Parsed_answer: a
----------------1-----------------
Generated Answer: Which of the following is a primary purpose of BGP?
Choose the correct answer:
a) Establishing communication between devices within a LAN
b) Exchanging routing information between autonomous systems
c) Optimizing local network traffic
d) Assigning IP addresses to devices

Answer: b) Exchanging routing information between autonomous systems

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used

In [3]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_fill_the_blank.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
filled_answers = []

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST]{question} \n Fill the blank: \n [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(f"Output: {generated_answer}")
    
    generated_answer = re.search(r'\[/INST\]\s+(.*)', generated_answer, re.DOTALL).group(1).strip()
    if "Sure" in generated_answer:
        generated_answer = generated_answer.split('\n', 1)[1]  # Skip the first line
    print(f"Generated Answer: {generated_answer}")
    generated_answers.append(generated_answer)
    
    # Find the position of the placeholder in the first string
    placeholder_position = question.find("________")

    # Extract the content filled in the placeholder
    filled_content = generated_answer[placeholder_position:placeholder_position + len("autonomous systems (ASes)")]
    filled_answers.append(filled_content)
    n_question += 1
    print(f"Filled content: {filled_content}")
    print(f"---------------------{n_question}-----------------------------")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP (Border Gateway Protocol) is a standardized exterior gateway protocol used to exchange routing and reachability information between Autonomous Systems (AS) on the Internet. It enables the selection of paths and routes for data to flow efficiently between networks.
Filled content:  Autonomous Systems (AS) 
---------------------1-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
In BGP, the AS-Path attribute represents the sequence of autonomous systems (ASes) that a route has traversed.

So, the correct answer is:

"sequence of autonomous systems (ASes)"
Filled content:  sequence of autonomous s
---------------------2-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

The correct answer is: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used to exchange routing information among autonomous systems (AS) on the internet. BGP route
Filled content: selection ______________ 
---------------------33-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP peering is used for establishing BGP session connections and exchanging routing information.
Filled content:  peering is used for esta
---------------------34-----------------------------
Generated Answer: The phenomenon where the Multi-Exit Discriminator (MED) value of a route frequently changes is referred to as "MED oscillations."

Therefore, the correct answer is:

MED oscillations.
Filled content:  as "MED oscillations."


---------------------35-----------------------------


: 